In [1]:
import time
import numpy as np
import pandas as pd

import requests

In [2]:
df = pd.read_excel(r'.\datasets\250716.xlsx')
print(df.head())

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   
3   4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294      경기도 과천시 중앙로 294   
4   5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56     대구광역시 수성구 무열로 56   

    중간집결 기한    최종도착 기한 이동방법  
0  0+1일 14시   0+1일 14시   개별  
1  0+28일 9시  0+28일 14시   집단  
2   0+3일 8시   0+3일 14시   집단  
3  0+1일 14시   0+1일 14시   개별  
4  0+3일 14시   0+3일 14시   개별  


In [3]:
num = np.array(df['순번'])

start = np.array(df['출발지 주소'])
mid = np.array(df['중간집결지 주소'])
end = np.array(df['최종 도착지 주소'])

In [ ]:
def geocoding(address: list):
    url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
    headers = {
        "x-ncp-apigw-api-key-id": "oa77i9oz1h",
        "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
        "Accept": "application/json"
    }

    params = {"query": address}
    resp = requests.get(url, headers = headers, params = params)
    resp.raise_for_status()
    data = resp.json()

    if data.get('addresses'):
        lat = float(data['addresses'][0]['y']) # 위도
        lng = float(data['addresses'][0]['x']) # 경도  
        return lat, lng
    
    else:
        lat = None
        lng = None
        return lat, lng

In [15]:
# geocoding
if __name__ == "__main__":
    chunk = 1_000
    start_index = 160_000
    last_index = 300_000
    
    for i in range(start_index, last_index, chunk):
        slc = slice(i, min(i + chunk, last_index))
        print(f'Processing records {slc.start} to {slc.stop}...')

        t0 = time.time()
        start_lat, start_lng = zip(*(geocoding(addr) for addr in start[slc]))
        mid_lat, mid_lng = zip(*(geocoding(addr) for addr in mid[slc]))
        end_lat, end_lng = zip(*(geocoding(addr) for addr in end[slc]))
        t1 = time.time()

        print(f'걸린시간(분): {(t1-t0)/60:.2f}')

        df_chunk = pd.DataFrame({'num': num[slc],
                       'start_lat': start_lat,
                       'start_lng': start_lng,
                       'mid_lat': mid_lat,
                       'mid_lng': mid_lng,
                       'end_lat': end_lat,
                       'end_lng': end_lng})
        
        mode = 'w' if i == start_index else 'a'
        header = True if i == start_index else False

        df_chunk.to_csv(r'.\results\transformed.csv', mode = mode, header = header, index = False)
        print(f'{slc.start} to {slc.stop} 저장 완료')

Processing records 160000 to 161000...
걸린시간(분): 4.93
160000 to 161000 저장 완료
Processing records 161000 to 162000...
걸린시간(분): 4.85
161000 to 162000 저장 완료
Processing records 162000 to 163000...
걸린시간(분): 5.06
162000 to 163000 저장 완료
Processing records 163000 to 164000...
걸린시간(분): 5.34
163000 to 164000 저장 완료
Processing records 164000 to 165000...
걸린시간(분): 5.28
164000 to 165000 저장 완료
Processing records 165000 to 166000...
걸린시간(분): 5.57
165000 to 166000 저장 완료
Processing records 166000 to 167000...
걸린시간(분): 5.55
166000 to 167000 저장 완료
Processing records 167000 to 168000...
걸린시간(분): 5.43
167000 to 168000 저장 완료
Processing records 168000 to 169000...
걸린시간(분): 4.58
168000 to 169000 저장 완료
Processing records 169000 to 170000...
걸린시간(분): 4.79
169000 to 170000 저장 완료
Processing records 170000 to 171000...
걸린시간(분): 4.58
170000 to 171000 저장 완료
Processing records 171000 to 172000...
걸린시간(분): 4.50
171000 to 172000 저장 완료
Processing records 172000 to 173000...
걸린시간(분): 5.27
172000 to 173000 저장 완료
Processing r